In [1]:
print('hello bitch')

hello bitch


In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn
from torch.nn import functional as F
from torchvision import transforms, models
from torchvision.transforms import v2 as T

import torchvision

# import lightning as pl
from transformers import (
    AutoProcessor,
    
    AutoModelForZeroShotImageClassification,
    CLIPModel,
    CLIPProcessor,
    CLIPProcessor,
)
from PIL import Image
import copy
import os
import subprocess
from torch.utils.data import Subset
import numpy as np
# from lightning.pytorch.callbacks import ModelCheckpoint
# models will use for transfer learning



clip_model = "openai/clip-vit-base-patch32"

processor: CLIPProcessor = AutoProcessor.from_pretrained(
    clip_model, use_fast=False, do_normalize=True, do_rescale=True
)  # try patch 14
clip: CLIPModel = AutoModelForZeroShotImageClassification.from_pretrained(clip_model)

2025-12-27 19:06:47.099010: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766862407.119745      98 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766862407.126039      98 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [ ]:
import tqdm
paths = [
    os.path.join(root, file) for root, dirs, files in os.walk("/kaggle/input/df2k-ost/train") for file in files
]


class ImageDataset(Dataset):
    def __init__(self, paths, transform=None):
        self.paths = paths
        self.transform = transform
        

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        image = processor(images=Image.open(path).convert("RGB"), return_tensors="pt").pixel_values.squeeze(0)
        return path, image


def filter_labels(paths, accepted_labels, non_accepted_labels):
    transform = transforms.Compose(
        [transforms.Resize((224, 224)), transforms.ToTensor()]
    )
    data = ImageDataset(paths)
    loader = DataLoader(data, batch_size=256, shuffle=False,num_workers=3)
    labels = accepted_labels + non_accepted_labels
    threshold = len(accepted_labels)  # if something more than this it is not accepted
    text_encodings = processor(
        text=labels, return_tensors="pt", padding=True
    )["input_ids"]
    
    filtered_paths = []
    print('text encodings shape:', text_encodings.shape)
    print('threshold:', threshold)
    clip.to(device)
    text_encodings = text_encodings.to(device)
    with torch.no_grad():
        loop = tqdm.tqdm(loader, total=len(loader))
        for paths, images in loop:
            images = images.to(device)
            outputs = clip(input_ids=text_encodings, pixel_values=images)
            logits = outputs.logits_per_image
            probs = logits.softmax(dim=-1)
            labels = probs.argmax(dim=1)
            labels = labels < threshold
            for i, path in enumerate(paths):
                if labels[i]:
                    filtered_paths.append(paths[i])
    return filtered_paths

device = 'cuda'
accepted = ['sky','landscape','mountains','flower','forest','sea','ocean','lake','nature','tree','trees','sunset','sunrise','river','beach','clouds','field','hill','grass','waterfall','mid_night']
non_accepted = ["car",'image','person','people','building','city','indoor','phone','computer','book','table']
paths = filter_labels(paths, accepted, non_accepted)

text encodings shape: torch.Size([32, 5])
threshold: 21


100%|██████████| 49/49 [04:47<00:00,  5.86s/it]


['/kaggle/input/df2k-ost/train/Flickr2K/002584.png',
 '/kaggle/input/df2k-ost/train/Flickr2K/001840.png',
 '/kaggle/input/df2k-ost/train/Flickr2K/000920.png',
 '/kaggle/input/df2k-ost/train/Flickr2K/002011.png',
 '/kaggle/input/df2k-ost/train/Flickr2K/000603.png',
 '/kaggle/input/df2k-ost/train/Flickr2K/001669.png',
 '/kaggle/input/df2k-ost/train/Flickr2K/001249.png',
 '/kaggle/input/df2k-ost/train/Flickr2K/000370.png',
 '/kaggle/input/df2k-ost/train/Flickr2K/002454.png',
 '/kaggle/input/df2k-ost/train/Flickr2K/001537.png',
 '/kaggle/input/df2k-ost/train/Flickr2K/000514.png',
 '/kaggle/input/df2k-ost/train/Flickr2K/001923.png',
 '/kaggle/input/df2k-ost/train/Flickr2K/001143.png',
 '/kaggle/input/df2k-ost/train/Flickr2K/000837.png',
 '/kaggle/input/df2k-ost/train/Flickr2K/000505.png',
 '/kaggle/input/df2k-ost/train/Flickr2K/002087.png',
 '/kaggle/input/df2k-ost/train/Flickr2K/001910.png',
 '/kaggle/input/df2k-ost/train/Flickr2K/002073.png',
 '/kaggle/input/df2k-ost/train/Flickr2K/002513

In [ ]:
import json
paths = None
with open('nature_paths.json','r') as f:
    paths = json.load(f)

In [ ]:
clip.get_ima

In [ ]:
import h5py
from sklearn.cluster import KMeans
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def store_embeddings(paths, target_dir):
    """
    Generate CLIP embeddings for images and store them in an h5 file.
    
    Args:
        paths: List of image file paths
        target_dir: Directory to store the h5 file
    """
    os.makedirs(target_dir, exist_ok=True)
    
    data = ImageDataset(paths)
    loader = DataLoader(data, batch_size=256, shuffle=False, num_workers=3)
    
    embeddings_list = []
    paths_list = []
    
    clip.to(device)
    
    with torch.no_grad():
        loop = tqdm.tqdm(loader, total=len(loader), desc="Generating embeddings")
        for batch_paths, images in loop:
            images = images.to(device)
            outputs = clip(pixel_values=images)
            image_embeddings = outputs.image_embeddings
            embeddings_list.append(image_embeddings.cpu().numpy())
            paths_list.extend(batch_paths)
    
    embeddings = np.concatenate(embeddings_list, axis=0)
    
    # Save to h5 file
    h5_path = os.path.join(target_dir, 'embeddings.h5')
    with h5py.File(h5_path, 'w') as f:
        f.create_dataset('embeddings', data=embeddings, compression='gzip')
        f.create_dataset('paths', data=[p.encode('utf-8') for p in paths_list], compression='gzip')
    
    print(f"Embeddings saved to {h5_path}")
    return h5_path


def filter_divergence(paths, k_means, embeddings_dir='embeddings'):
    """
    Filter paths using k-means clustering on embeddings to select diverse samples.
    
    Args:
        paths: List of image file paths
        k_means: Number of clusters or a pre-trained KMeans object
        embeddings_dir: Directory containing the embeddings h5 file
    
    Returns:
        List of filtered paths (one per cluster)
    """
    h5_path = os.path.join(embeddings_dir, 'embeddings.h5')
    
    with h5py.File(h5_path, 'r') as f:
        embeddings = f['embeddings'][:]
        stored_paths = [p.decode('utf-8') for p in f['paths'][:]]
    
    # Create or use provided KMeans
    if isinstance(k_means, int):
        n_clusters = k_means
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        kmeans.fit(embeddings)
    else:
        kmeans = k_means
        n_clusters = kmeans.n_clusters
    
    # Get cluster assignments
    labels = kmeans.labels_
    
    # Select one representative from each cluster (the one closest to cluster center)
    filtered_paths = []
    for cluster_id in range(n_clusters):
        cluster_indices = np.where(labels == cluster_id)[0]
        if len(cluster_indices) > 0:
            # Find the index closest to cluster center
            cluster_embeddings = embeddings[cluster_indices]
            distances = np.linalg.norm(cluster_embeddings - kmeans.cluster_centers_[cluster_id], axis=1)
            closest_idx = cluster_indices[np.argmin(distances)]
            filtered_paths.append(stored_paths[closest_idx])
    
    print(f"Filtered {len(stored_paths)} paths to {len(filtered_paths)} diverse samples")
    return filtered_paths


12434

In [19]:
'what is the meaning of life'

'what is the meaning of life'

In [18]:
import json
import random
random_paths = random.shuffle(paths)
paths_clean = {'paths': random_paths}
with open('nature_paths.json','w') as f:
    json.dump(paths_clean,f)